In [3]:
import tensorflow as tf 
print(tf.__version__)

2.7.0


In [4]:
import matplotlib
print(matplotlib.__version__)

3.5.0


In [5]:
import numpy as np 
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [6]:
datasetFileName = "numandataset.npz"
with np.load(datasetFileName, allow_pickle=True) as data:
    dataImages = data['images']
    dataLabels = data['labels']
    dataLabelNames = data['labelnames']
    
print(tf.__version__)
print(dataImages.shape)
print(dataLabels)
print(dataLabelNames)

2.7.0
(36,)
[0 1 2 3 4 5 6 7 8 1 2 4 6 7 8 1 2 3 4 5 6 7 0 1 4 5 6 7 8 0 1 4 5 6 7 8]
['afiq' 'azureen' 'gavin' 'goke' 'inamul' 'jincheng' 'mahmuda' 'numan'
 'saseendran' 'azureen' 'gavin' 'inamul' 'mahmuda' 'numan' 'saseendran'
 'azureen' 'gavin' 'goke' 'inamul' 'jincheng' 'mahmuda' 'numan' 'afiq'
 'azureen' 'inamul' 'jincheng' 'mahmuda' 'numan' 'saseendran' 'afiq'
 'azureen' 'inamul' 'jincheng' 'mahmuda' 'numan' 'saseendran']


In [7]:
classNames = sorted(np.unique(dataLabelNames))
print(classNames)



['afiq', 'azureen', 'gavin', 'goke', 'inamul', 'jincheng', 'mahmuda', 'numan', 'saseendran']


In [8]:
import cv2 as cv

N = len(dataImages)
shape = (N, 200,200,3)

y = np.empty(shape, dtype='uint8')

for i in range(N):
    y[i] = cv.resize(dataImages[i], [200,200], interpolation=cv.INTER_NEAREST)
    
dataImages.dtype, y.dtype, y.shape

dataImages = y

trainImages, testImages, trainLabels, testLabels = train_test_split(dataImages, dataLabels, test_size=0.30, random_state=42)
print(trainImages.shape)
print(testImages.shape)

(25, 200, 200, 3)
(11, 200, 200, 3)


In [9]:
import numpy as np

print(trainImages[0].shape)
print(trainImages[1].shape)

(200, 200, 3)
(200, 200, 3)


In [10]:
for i in range(len(trainImages)):
    trainImages[i] = cv.resize(trainImages[i], (200,200), interpolation=cv.INTER_NEAREST)



In [11]:
inputShape = trainImages[0].shape
outputShape = len(classNames)

inputShape, outputShape

((200, 200, 3), 9)

In [12]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape = inputShape),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(outputShape)

])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 120000)            0         
                                                                 
 dense (Dense)               (None, 128)               15360128  
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                                 
 dense_2 (Dense)             (None, 128)               16512     
                                                                 
 dense_3 (Dense)             (None, 9)                 1161      
                                                                 
Total params: 15,394,313
Trainable params: 15,394,313
Non-trainable params: 0
_________________________________________________________________


In [13]:
model.compile(optimizer='adam', loss= tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

In [14]:
model.fit(trainImages, trainLabels, epochs=20)

Epoch 1/20
1/1 [==============================] - 3s 3s/step - loss: 134.4619 - accuracy: 0.2000
Epoch 2/20
1/1 [==============================] - 0s 310ms/step - loss: 3055.6897 - accuracy: 0.0800
Epoch 3/20
1/1 [==============================] - 0s 216ms/step - loss: 5103.6108 - accuracy: 0.2400
Epoch 4/20
1/1 [==============================] - 0s 250ms/step - loss: 4492.7837 - accuracy: 0.1600
Epoch 5/20
1/1 [==============================] - 0s 294ms/step - loss: 4255.1426 - accuracy: 0.1200
Epoch 6/20
1/1 [==============================] - 0s 247ms/step - loss: 3508.2373 - accuracy: 0.2000
Epoch 7/20
1/1 [==============================] - 0s 237ms/step - loss: 2034.5437 - accuracy: 0.2800
Epoch 8/20
1/1 [==============================] - 0s 279ms/step - loss: 2793.7100 - accuracy: 0.1200
Epoch 9/20
1/1 [==============================] - 0s 242ms/step - loss: 1637.2837 - accuracy: 0.2400
Epoch 10/20
1/1 [==============================] - 0s 310ms/step - loss: 1980.6077 - accuracy: 

In [21]:
testLoss, testAcc = model.evaluate(testImages, testLabels, verbose=2)

print('\nTest accuracy:', testAcc)

predictions = model.predict(testImages)
predictions[0]

1/1 - 0s - loss: 1742.8153 - accuracy: 0.4545 - 260ms/epoch - 260ms/step

Test accuracy: 0.4545454680919647


array([  4800.4185,   2985.057 , -13125.918 ,   6043.956 ,  -1863.7476,
        -3312.5935,   2944.5718,   4752.2627,   7945.532 ], dtype=float32)

In [24]:
def plot_image(i, predictions_array, true_label, img):
  true_label, img = true_label[i], img[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])

  plt.imshow(img, cmap=plt.cm.binary)

  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = 'yellow'
  else:
    color = 'red'

  plt.xlabel("{} {:2.0f}% ({})".format(classNames[predicted_label],
                                100*np.max(predictions_array),
                                classNames[true_label]),
                                color=color)

In [25]:
for i in range (len(predictions)):
    plt.figure(figsize=(6,6))
    plot_image(i, predictions[i], testLabels, testImages)


plt.show()

NameError: name 'plt' is not defined